## Elliptic curves and ECDSA implementation

|               |                          |
|:--------------|:-------------------------|
|Author         |P. GARREAU                |
|Status         |Finished                  |
|Project        |Elliptic curves and ECDSA |

### Table of Content

1. [Constants and conversions](#1-Constants-and-conversions)  
2. [Operations on elliptics curves points with affine coordinates](#2-Operations-on-elliptics-curves-points-with-affine-coordinates)  
3. [Operations on elliptics curves points with jacobian coordinates](#3-Operations-on-elliptics-curves-points-with-jacobian-coordinates)  
4. [Quick exponenciation](#4-Quick-exponenciation)  
5. [Scalar mulitiplication of an elliptic curve's point](#5-Scalar-mulitiplication-of-an-elliptic-curve's-point)  
6. [Montgomery scale](#6-Montgomery-scale)  
7. [Diffie-Hellman algorithm](#7-Diffie-Hellman-algorithm)
8. [ECDSA (Elliptic Curve Digital Signature Algorithm)](#8-ECDSA-(Elliptic-Curve-Digital-Signature-Algorithm))  
   8.1. [Hash function](#81-Hash-function)  
   8.2. [Keys generation](#82-Keys-generation)  
   8.3. [Signature](#83-Signature)  
   8.4. [Verification](#84-Verification)  

### 1. Constants and conversions

In [127]:
import random

# CONSTANTES
a = 0
b = 3
p = 7
n = 13

# CONVERSIONS
def invModulaire(a, p):
    return pow(a, p - 2, p)

def fromCartesianToJacobian(P, z):
    return ((P[0] * (z ** 2)) % p, (P[1] * (z ** 3)) % p, z)

def fromJacobianToCartesian(P):
    invZ = invModulaire(P[2], p)
    return ((P[0] * (invZ ** 2)) % p, (P[1] * (invZ ** 3)) % p)

### 2. Operations on elliptics curves points with affine coordinates

- Modular inverse
- Addition
- Doubling
- Point on the curve

In [128]:
def addition(P, Q, p):
    lam = ((Q[1] - P[1]) * invModulaire(Q[0] - P[0], p)) % p
    x_3 = (lam ** 2 - P[0] - Q[0])
    y_3 = (lam * (P[0] - x_3) - P[1])
    return (x_3 % p, y_3 % p)

def doublement(P, p, a):
    lam = ((3 * (P[0] ** 2) + a) * invModulaire(2 * P[1], 7))
    x_2 = (lam ** 2 - 2 * P[0]) % p
    y_2 = (lam * (P[0] - x_2) - P[1]) % p
    return (x_2, y_2)

def onTheCurve(P, p, a, b):
    return ((P[1] ** 2) % p == (P[0] ** 3 + a * P[0] + b) % p)

P = (1, 5)
Q = (4, 5)
add = addition(P, Q, p)
print("[CARTESIAN-] P + Q = " + str(add))
double = doublement(P, p, a)
print("[CARTESIAN-]    2P = " + str(double))

[CARTESIAN-] P + Q = (2, 2)
[CARTESIAN-]    2P = (6, 4)


### 3. Operations on elliptics curves points with jacobian coordinates

- Addition
- Doubling
- Point on the curve

In [129]:
def addition_jacob(P, Q, p):
    Z1Z1 = P[2] ** 2
    Z2Z2 = Q[2] ** 2
    U1 = P[0] * Z2Z2
    U2 = Q[0] * Z1Z1
    S1 = P[1] * Q[2] * Z2Z2
    S2 = Q[1] * P[2] * Z1Z1
    H = U2 - U1
    I = (2 * H) ** 2
    J = H * I
    r = 2 * (S2 - S1)
    V = U1 * I
    X3 = (r ** 2 - J - 2 * V) % p
    return (X3, (r * (V - X3) - 2 * S1 * J) % p, (((P[2] + Q[2]) ** 2 - Z1Z1 - Z2Z2) * H) % p)

def doublement_jacob(P, p):
    A = P[0] ** 2
    B = P[1] ** 2
    C = B ** 2
    D = 2 * ((P[0] + B) ** 2 - A - C)
    E = 3 * A
    F = E ** 2
    X3 = (F - 2 * D) % p
    return (X3, (E * (D - X3) - 8 * C) % p, (2 * P[1] * P[2]) % p)

def onTheCurveJacob(P, p, a, b):
    return ((P[1] ** 2) % p == (P[0] ** 3 + a * P[0] * (P[2] ** 4) + b * (P[2] ** 6)) % p)

P_tilde = (5, 3, 4)
Q_tilde = (1, 2, 1)
add_tilde = addition_jacob(P_tilde, Q_tilde, p)
print("[JACOBIAN--] P + Q = " + str(add_tilde))
double_tilde = doublement_jacob(P_tilde, p)
print("[JACOBIAN--]    2P = " + str(double_tilde))

[JACOBIAN--] P + Q = (4, 2, 4)
[JACOBIAN--]    2P = (1, 2, 3)


### 4. Quick exponenciation

In [130]:
def valueBit(x, i):
    return (x >> i) & 1

def nbBits(x):
    return len(bin(x)[2:])

def exponentiation(a, k, p):
    n = nbBits(k)
    result = 1
    for i in range(0, n):
        if (valueBit(k, n - i) == 1):
            result = result * a
        result = result ** 2
    return result % p

def exponentiation_2(a, k, p):
    n = nbBits(k)
    result = 1
    for i in range(0, n):
        result = (result ** 2) * a ** (k >> n - i) & 1
    return result % p

### 5. Scalar mulitiplication of an elliptic curve's point

In [131]:
def multiplication_scalaire(P, k, p):
    n = nbBits(k)
    result = P
    for i in range(0, n - 1):
        result = doublement_jacob(result, p)
        if (valueBit(k, n - i) == 1):
            result = addition_jacob(result, P, p)
    return result

P = (1, 2, 1)
k = 3
multiplication_3 = multiplication_scalaire(P, k, p)
print("[JACOBIAN--] " + str(k) + "P = " + str(multiplication_3))

[JACOBIAN--] 3P = (5, 3, 4)


### 6. Montgomery scale

In [132]:
def echelleMontgomery(P, k, p):
    n = nbBits(k)
    R = [P, doublement(P, p, a)]
    for i in range(n - 2, -1, -1):
        b = valueBit(k, i)
        R[1 - b] = addition(R[1 - b], R[b], p)
        R[b] = doublement(R[b], p, a)
    return R[0]

def echelleMontgomeryJacob(P, k, p):
    n = nbBits(k)
    R = [P, doublement_jacob(P, p)]
    for i in range(n - 2, -1, -1):
        b = valueBit(k, i)
        R[1 - b] = addition_jacob(R[1 - b], R[b], p)
        R[b] = doublement_jacob(R[b], p)
    return R[0]

P = (6, 4, 1)
k = 6
R_returned = echelleMontgomeryJacob(P, k, p)
print("[JACOBIAN--] " + str(k) + "P = " + str(R_returned))
print("[CARTESIAN-] " + str(k) + "P = " + str(fromJacobianToCartesian(R_returned)))

[JACOBIAN--] 6P = (2, 2, 4)
[CARTESIAN-] 6P = (1, 2)


### 7. Diffie-Hellman algorithm

In [133]:
def generateRandomInt(p):
    return random.randint(1, p - 1)

In [134]:
def DiffieHellman(P, p, a, b, k):
    n1 = generateRandomInt(k)
    print("[ALICE-] Génération d'un nombre aléatoire entre 1 et " + str(k) + " : n1 = " + str(n1))
    n2 = generateRandomInt(k)
    print("[BOB---] Génération d'un nombre aléatoire entre 1 et " + str(k) + " : n2 = " + str(n2))
    A = echelleMontgomery(P, n1, p)
    print("[ALICE-] Calcul de A = n1 * P = " + str(A))
    B = echelleMontgomery(P, n2, p)
    print("[BOB---] Calcul de B = n2 * P = " + str(B))
    return (echelleMontgomery(B, n1, p), echelleMontgomery(A, n2, p))

P = (5, 3)
k = 13
print("[ALICE-] Création d'une clé pour une communication avec Bob")
print("[ALICE-] Envoi de P = " + str(P) + ", point d'ordre k = " + str(k) + " qui appartient à la courbe elliptique (a, b) = (" + str(a) + ", " + str(b) + ")")
(C1, C2) = DiffieHellman(P, p, a, b, k)
print("[ALICE-] Calcul de C1 = " + str(C1))
print("[BOB---] Calcul de C2 = " + str(C2))

[ALICE-] Création d'une clé pour une communication avec Bob
[ALICE-] Envoi de P = (5, 3), point d'ordre k = 13 qui appartient à la courbe elliptique (a, b) = (0, 3)
[ALICE-] Génération d'un nombre aléatoire entre 1 et 13 : n1 = 2
[BOB---] Génération d'un nombre aléatoire entre 1 et 13 : n2 = 10
[ALICE-] Calcul de A = n1 * P = (1, 5)
[BOB---] Calcul de B = n2 * P = (3, 4)
[ALICE-] Calcul de C1 = (2, 2)
[BOB---] Calcul de C2 = (2, 2)


### 8. ECDSA (Elliptic Curve Digital Signature Algorithm)

#### Signature Protocol
- Alice:
  - Key generation:
    - Choose random number s in [1, n-1]
    - Compute sG = Q
    - Public key: Q
    - Private key: s
  - Signature:
    - Choose random number k in [1, n-1]
    - Compute kG = (i,j)
    - Compute x = i % n
    - Compute y = (1/k)*(H(m) + s * x) % n
    - Signature: (x, y)
- Bob:
  - Verification:
    - Q != O and Q on the curve
    - nQ == O
    - (x, y) in [1, n-1]²
    - x = i % n



#### 8.1. Hash function

In [135]:
def hashFunction(m):
    returned_message = ""
    for i in range(0, len(m)):
        returned_message += str(ord(m[i]))
    return int(returned_message)

#### 8.2. Keys generation

In [136]:
print("[------] Génération des clés")
# Choosing s in [1, n-1] :
s = generateRandomInt(n)
# Computing Q using G = (5, 3):
G = fromCartesianToJacobian((6, 4), 1)
Q = echelleMontgomeryJacob(G, s, p)
print("[ALICE-] Clé publique Q = " + str(fromJacobianToCartesian(Q)) + " et clé privée : s = " + str(s))

[------] Génération des clés
[ALICE-] Clé publique Q = (5, 4) et clé privée : s = 3


#### 8.3. Signature

In [137]:
print("[------] Signature")
m = "J'aime ce module"
x = 0
y = 0
while (x == 0 or y == 0):
    # Choosing k in [1, n-1] :
    k = generateRandomInt(n)
    # Computing (i, j) using G = (5, 3) and k:
    P = echelleMontgomeryJacob(G, k, p)
    (i, j) = fromJacobianToCartesian(P)
    x = i % n
    inv_k = invModulaire(k, n)
    y = (inv_k * (hashFunction(m) % n + s * x)) % n

print("[ALICE-] (x, y) = (" + str(x) + ", " + str(y) + ") and k = " + str(k))

[------] Signature
[ALICE-] (x, y) = (6, 2) and k = 12


#### 8.4. Verification

In [138]:
print("[------] Verification")
# Q on the curve verification:
if (onTheCurve(Q, p, a, b)):
    print("[BOB---] OK ! Q on the curve")
else:
    print("[BOB---] Not OK ! Q not on the curve")

# nQ equal to infinite point:
infinitePoint = (1, 1, 0)
nQ = echelleMontgomeryJacob(Q, n, p)
print("[BOB---] nQ = " + str(nQ))
if (nQ[2] == 0):
    print("[BOB---] OK ! nQ = O")
else:
    print("[BOB---] Not OK ! nQ != O")

# x and y in [1, n-1]:
if (1 <= x <= n - 1 and 1 <= y <= n - 1):
    print("[BOB---] OK ! (x,y) appartiennent à [1, n-1]")
else:
    print("[BOB---] Not OK ! (x,y) n'appartiennent pas à [1, n-1]")

inv_y = invModulaire(y, n)
inter1 = echelleMontgomeryJacob(G, ((hashFunction(m) * inv_y) % n), p)
inter2 = echelleMontgomeryJacob(Q, ((x * inv_y) % n), p)
P = addition_jacob(inter1, inter2, p)
P_cartesian = fromJacobianToCartesian(P)
print("[BOB---] (i, j) = " + str(P_cartesian))

# x = i % p
if (x == (P_cartesian[0] % n)):
    print("[BOB---] OK ! Verification réussie")
else:
    print("[BOB---] Not OK ! x différent de i modulo p")

[------] Verification
[BOB---] OK ! Q on the curve
[BOB---] nQ = (1, 1, 0)
[BOB---] OK ! nQ = O
[BOB---] OK ! (x,y) appartiennent à [1, n-1]
[BOB---] (i, j) = (6, 3)
[BOB---] OK ! Verification réussie
